In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import download_posts

In [ ]:
POSTS_DIR = "posts"


soup = download_posts.fetch_post_as_soup("23291")

#download_posts.save_posts(data_dir="posts")

In [ ]:
soup

In [ ]:
!pip install selenium

In [ ]:
import os
original_path = os.environ["PATH"]

In [ ]:
# See https://stackoverflow.com/questions/46753393/how-to-make-firefox-headless-programatically-in-selenium-with-python

extra_paths = [
    "/home/jcfr/Software/firefox-57.0/",
    "/home/jcfr/Projects/kwblog-data"
]

os.environ["PATH"] = ":".join(extra_paths) + ":" + original_path

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
options.set_headless(headless=True)
driver = webdriver.Firefox(firefox_options=options)
driver.implicitly_wait(1)
print("Firefox Headless Browser Invoked")

      

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#my_url = "https://blog.kitware.com/avogadro-1-91-released/"
my_url = "https://blog.kitware.com/slicervirtualreality/"


driver.get(my_url)

element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "post-views-count"))
)


print(element)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from joblib import Parallel, delayed

import download_posts


last_page=0

r = Parallel(n_jobs=10, verbose=10)(
    delayed(download_posts.save_posts)(
        last_page=page_number,
        first_page=page_number,
        data_dir="posts"
    )
    for page_number in download_posts.page_range(last_page=last_page)
)

#Parallel(n_jobs=N_JOBS, backend='threading')(
#        delayed(handle_item)(item, newFolder, depth + 1, bc) for item in items)

In [2]:
%reload_ext autoreload
%autoreload 2

import download_posts as dp
#soup = dp.fetch_post_as_soup(13304)

In [3]:
import glob
import os

from bs4 import BeautifulSoup

#hosts = ["midas3.kitware.com", "midas.kitware.com"]
hosts = ["visomics.org"]

references_per_year = {}

for html_filepath in glob.glob('posts/*.html'):
    metadata_filepath = os.path.splitext(html_filepath)[0] + ".json"
    metadata = dp.load_metadata(metadata_filepath)

    year = metadata["date"].split(",")[1].strip()   
    metadata["year"] = year
    
    with open(html_filepath) as content:
        soup = BeautifulSoup(content, 'html.parser')
    
    references = []
    
    for host in hosts:
        for a_tag in soup.select('a[href*="%s"]' % host):
            references.append((a_tag.text.strip(), a_tag["href"],))
    
    if references:
        if year not in references_per_year:
            references_per_year[year] = {}

        year_references = references_per_year[year]

        year_references[metadata["id"]] = {"references": references, "metadata": metadata}
    

In [4]:
for year in sorted(references_per_year.keys()):
    year_references = references_per_year[year]
    print("\n## %s" % year)
    for post_id, post_info in year_references.items():
        metadata = post_info["metadata"]
        references = post_info["references"]
        print("\n### [{title}](https://blog.kitware.com/?p={id}) (post_id: {id})".format(**metadata))
        for reference in references:
            if reference[0] == reference[1]:
                print("* [ ] %s" % reference[0])
            else:
                print("* [ ] %s: %s" % reference)


## 2012

### [January 2012 – Recent Releases](https://blog.kitware.com/?p=13267) (post_id: 13267)
* [ ] www.visomics.org: http://www.visomics.org

### [SPIE Medical Imaging 2012](https://blog.kitware.com/?p=12866) (post_id: 12866)
* [ ] http://www.visomics.org/

### [Kitware Introduces Visomics for Bioinformatics](https://blog.kitware.com/?p=11951) (post_id: 11951)
* [ ] http://www.visomics.org: http://www.visomics.org/

### [Future of Medicine: Information Aggregation and Visualization](https://blog.kitware.com/?p=12869) (post_id: 12869)
* [ ] Visomics: http://www.visomics.org/

## 2013

### [ITK Workshop at the University of Houston](https://blog.kitware.com/?p=12710) (post_id: 12710)
* [ ] Visomics: http://www.visomics.org/

## 2014

### [VIZBI Tutorial: Open Source Visualization, Analytics, and Informatics Tools](https://blog.kitware.com/?p=12615) (post_id: 12615)
* [ ] Visomics: http://www.visomics.org/


In [29]:
import download_posts

In [30]:


#download_posts.list_post_with_default_feature_image()

download_posts.parallel_list_post_with_default_feature_image(last_page=1)



[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:    1.7s finished
